# Collecting and analysing real-time data from Swiss charging stations

## For details of data see:

Summary: https://www.uvek-gis.admin.ch/BFE/diemo/demo-prod.html<br>
Map:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; https://map.geo.admin.ch/?lang=de&topic=ech&bgLayer=ch.swisstopo.pixelkarte-farbe&layers=ch.bfe.ladestellen-elektromobilitaet

## Libraries and settings

In [ ]:
# Libraries
from __future__ import print_function

import os
import json
import fnmatch
import numpy as np
import pandas as pd
import urllib.request
import matplotlib.pyplot as plt
from pandas import json_normalize
from datetime import datetime, timedelta

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show current working directory
print(os.getcwd())

## Server query of utilization data

In [ ]:
# Function
def my_func():
    
    # Get current time
    # print(datetime.now().strftime('%Y-%m-%d-%H-%M-%S'))
    
    try:
        
        # Server request to get .json-file
        url = ('https://data.geo.admin.ch/ch.bfe.ladestellen-elektromobilitaet/'\
               'status/oicp/ch.bfe.ladestellen-elektromobilitaet.json')
        request = urllib.request.urlopen(url).read()
        data = json.loads(request)
        
        # Path on Windows with filename based on the current time
        filename = 'data-%s.json'% datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
        # Path on Linux-Server with filename based on the current time
        #filename = '/home/ubuntu/data/data-%s.json'%datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
                       
        # Saving the file
        with open(filename, 'w') as outfile:
            json.dump(data, outfile)
        
    except:

        pass

    return datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Function call
dt = my_func()
dt

## Analysing the utilization rate of Electric Vehicle Supply Equipment (EVSE) units

In [ ]:
# Get .json files
flist = fnmatch.filter(os.listdir('.'), '*.json')

# Read to data frame
df_orig = pd.read_json(flist[0])
df = json_normalize(df_orig['EVSEStatuses'], record_path=['EVSEStatusRecord'])
df

## Showing the number of EVSEs per EVSEStatus

In [ ]:
# Show number of EVSE per EVSEStatus
df_bar = df['EVSEStatus'].value_counts().sort_values(ascending=True)
print(df_bar)

# Simple bar chart
ncars = list(df_bar.values)
make = list(df_bar.index.values)
y_pos = np.arange(len(make))
# print(ncars)
# print(bars)

# Figure
fig, ax = plt.subplots(figsize=(6,4))
ax.barh(y_pos, ncars, align='center', color=['red', 'gray', 'gray', 'green'], alpha=0.8)
ax.set_yticks(y_pos, labels=make)
ax.set_xlabel('Number of EVSE')
ax.set_ylabel('EVSE-Status')
ax.set_title(f'Utilization of EVSE ({dt})')
ax.grid()

# Show graph
plt.show()


## Calculating the utilization rate of EVSEs

In [ ]:
f"Utilization rate ({dt}): {df_bar[0] / (df_bar[0] + df_bar[3]) * 100:.2} %"

## Removing .json files from the folder

In [ ]:
# Remove all .json files from the folder
for file in flist:
    try:
        os.remove(file)
        print("The following .json file was deleted:", file)
    except:
        print("Error while deleting the .json file:", file)

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')